## Setting up environment

In [1]:
import os                                    # system functions

import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.fsl as fsl          # fsl
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.algorithms.modelgen as model   # model generation
import nipype.algorithms.rapidart as ra      # artifact detection

# Extra interfaces (not used for now!)
import nipype.interfaces.matlab as matlab
import nipype.interfaces.freesurfer as fs

# Useful functions
from os.path import join as opj
import glob
from nipype.interfaces.base import Bunch
from numpy import diff

# Debugging
from nipype import config
config.enable_debug_mode() # Debug mode on
cfg = dict(loggging = dict(workflow_level = 'DEBUG'),
          execution ={'stop_on_first_crash': True,
                     'hash_method': 'content'})
config.update_config(cfg)

# Set up environment
wd = os.getcwd()
subjdir = opj(wd, 'kalanit_fsl')
fs_dir = '/Users/sll-members/fmri/software/freesurfer/subjects'
matlab.MatlabCommand.set_default_matlab_cmd('/Applications/MATLAB_R2012b.app/bin/matlab -nodesktop -nosplash')
fs.FSCommand.set_default_subjects_dir(fs_dir)

# Define important files
anat_file = opj(subjdir, '3Danatomy', 't1.nii')
bold_files = [opj(subjdir, f) for f in os.listdir(subjdir) if 'BOLD' in f]
inplane_file = glob.glob(opj(subjdir, '*Inplane*'))[0]

print wd

/Users/sll-members/fmri/Psych_204b


## Preprocessing/modeling pipeline

### Preprocessing workflow

In [2]:
# Import workflows
from nipype.workflows.fmri.fsl.preprocess import create_fsl_fs_preproc
from nipype.workflows.fmri.fsl.estimate import create_modelfit_workflow
from nipype.workflows.fmri.fsl.estimate import create_fixed_effects_flow

# Reorient to standard
reorient = pe.MapNode(fsl.Reorient2Std(), iterfield = ['in_file'], name = 'reorient')

# Get total length of image file
def gettsize(func):
    from nibabel import load
    _,_,_,timepoints = load(func).get_shape()
    tsize = timepoints - 4
    return tsize

tsize = pe.MapNode(util.Function(input_names = ['func'],
                                 output_names = ['tsize'],
                                 function = gettsize),
                   iterfield = ['func'],
                   name = 'tsize')


# Clip first 4 frames
fslroi = pe.MapNode(fsl.ExtractROI(t_min = 4), 
                    iterfield = ['in_file', 't_size'], 
                    name = 'fslroi')

# Preprocessing
preproc = create_fsl_fs_preproc(whichvol = 'middle')
preproc.base_dir = opj(subjdir, 'tmp')
preproc.inputs.inputspec.fwhm = 0
preproc.inputs.inputspec.highpass = 0
preproc.inputs.inputspec.subject_id = 'kalanit'
preproc.inputs.inputspec.subjects_dir = fs_dir

# Artifact detection
art = pe.Node(ra.ArtifactDetect(parameter_source = 'FSL',
                               mask_type = 'file',
                               norm_threshold = 1,
                               zintensity_threshold = 3),
             name = 'art')

### Subject info (for modeling)

In [3]:
# List of contrasts
condition_names = ['adult', 'child', 'body', 'limb',
                   'car', 'instrument', 'corridor',
                   'house', 'word', 'number'];

contrast_list = [('adult', 'T', condition_names, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
                ('child', 'T', condition_names, [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
                ('body', 'T', condition_names, [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
                ('limb', 'T', condition_names, [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
                ('car', 'T', condition_names, [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
                ('instrument', 'T', condition_names, [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
                ('corridor', 'T', condition_names, [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
                ('house', 'T', condition_names, [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
                ('word', 'T', condition_names, [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
                ('number', 'T', condition_names, [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
                ('faces > places', 'T', condition_names, [1, 1, 0, 0, 0, 0, -1, -1, 0, 0]),
                ('places > faces', 'T', condition_names, [-1, -1, 0, 0, 0, 0, 1, 1, 0, 0]),
                ('bodies > objects', 'T', condition_names, [0, 0, 1, 1, -1, -1, 0, 0, 0, 0]),
                ('animate > inanimate', 'T', condition_names, [1, 1, 1, 1, -1, -1, -1, -1, 0, 0]),
                ('inanimate > animate', 'T', condition_names, [-1, -1, -1, -1, 1, 1, 1, 1, 0, 0]),
                ]

# Create subject info
model_files = glob.glob(opj(subjdir, 'Model', '*.txt'))
subjectinfo = []

for r in range(len(model_files)):
    # Read model info
    model_f = open(model_files[r], 'r')
    model_str = model_f.read()
    model_info = [s.split('\t') for s in model_str.split('\r')]
    model_f.close()

    # Clean up model_info
    for i in range(len(model_info)):
        model_info[i][0] = int(model_info[i][0])

    # Get onsets
    all_onsets = [m[0] for m in model_info]
    run_dur = [list(set(diff(all_onsets)))]
    cond_durs = run_dur*len(condition_names)

    # Split onsets by condition
    cond_onsets = [[m[0] for m in model_info if m[2] == c] for c in condition_names]

    # Create Bunch object
    subjectinfo.insert(r, Bunch(conditions = condition_names,
                                onsets = cond_onsets,
                                durations = cond_durs))

### Modeling

In [4]:
# Specify model
modelspec = pe.Node(model.SpecifyModel(input_units = 'secs',
                                      time_repetition = 2.0,
                                      high_pass_filter_cutoff = 0,
                                      subject_info = subjectinfo), 
                    name = 'modelspec')

# First level modeling
lvl1model = create_modelfit_workflow()
lvl1model.inputs.inputspec.interscan_interval = 2.0
lvl1model.inputs.inputspec.contrasts = contrast_list
lvl1model.inputs.inputspec.model_serial_correlations = True
lvl1model.inputs.inputspec.bases = {'dgamma':{'derivs': False}}

# Fixed effects modeling
fixedfx = create_fixed_effects_flow()

### Input/output specification

In [5]:
# Create input node
inputnode = pe.Node(util.IdentityInterface(fields = ['func', 'struct']),
                   name = 'inputspec')
inputnode.inputs.func = bold_files
inputnode.inputs.struct = anat_file

# Create datasink
datasink = pe.Node(nio.DataSink(base_directory = subjdir,
                               container = 'result'),
                  name = 'datasink')

### Connect preprocessing and first-level workflows

In [6]:
lv1_workflow = pe.Workflow(name = 'metaflow')
lv1_workflow.base_dir = opj(subjdir, 'tmp')

# TODO: lvl1model missing functional data?
lv1_workflow.connect([(inputnode, reorient, [('func', 'in_file')]),
                      (reorient, tsize, [('out_file', 'func')]),
                      (reorient, fslroi, [('out_file', 'in_file')]),
                      (tsize, fslroi, [('tsize', 't_size')]),
                      (fslroi, preproc, [('roi_file', 'inputspec.func')]),
                      (preproc, art, [('outputspec.motion_parameters',
                                       'realignment_parameters'),
                                      ('outputspec.realigned_files', 'realigned_files'),
                                      ('outputspec.mask_file', 'mask_file')]),
                      (preproc, modelspec, [('outputspec.highpassed_files',
                                             'functional_runs')]),
                      (art, modelspec, [('outlier_files', 'outlier_files')]),
                      (modelspec, lvl1model, [('session_info', 'inputspec.session_info')]),
                      (preproc, lvl1model, [('outputspec.highpassed_files',
                                             'inputspec.functional_data')]),
                      (preproc, fixedfx, [('outputspec.mask_file', 'flameo.mask_file')]),
                 ])

### Connect first-level and fixed effect workflows

In [7]:
def sort_copes(files):
    numelements = len(files[0])
    outfiles = []
    for i in range(numelements):
        outfiles.insert(i,[])
        for j, elements in enumerate(files):
            outfiles[i].append(elements[i])
    return outfiles

def sort_varcopes(files):
    numelements = len(files[0])
    outfiles = []
    for i in range(numelements):
        outfiles.insert(i,[])
        for j, elements in enumerate(files):
            outfiles[i].append(elements[i])
    return outfiles

copesort = pe.Node(util.Function(input_names = ['files'],
                                 output_names = ['outfiles'],
                                 function = sort_copes),
                   name = 'copesort')

varcopesort = pe.Node(util.Function(input_names = ['files'],
                                    output_names = ['outfiles'],
                                    function = sort_varcopes),
                      name = 'varcopesort')

def num_copes(files):
    return len(files)

ncopes = pe.Node(util.Function(input_names = ['files'],
                               output_names = ['num_copes'],
                               function = num_copes),
                 name = 'ncopes')

pickfirst = lambda x : x[0]
first = pe.Node(util.Function(input_names = ['x'],
                              output_names = ['first_file'],
                              function = pickfirst),
                name = 'first')

#(preproc, first, [('outputspec.mask_file', 'x')]),
                      #(first, fixedfx, [('first_file', 'flameo.mask_file')]),
lv1_workflow.connect([(lvl1model, ncopes, [('outputspec.copes', 'files')]),
                      (lvl1model, copesort, [('outputspec.copes', 'files')]),
                      (lvl1model, varcopesort, [('outputspec.varcopes', 'files')]),
                      (lvl1model, fixedfx, [('outputspec.dof_file',
                                            'inputspec.dof_files')]),
                      (copesort, fixedfx, [('outfiles', 'inputspec.copes')]),
                      (varcopesort, fixedfx, [('outfiles', 'inputspec.varcopes')]),
                      (ncopes, fixedfx, [('num_copes', 'l2model.num_copes')])
                     ])

### Create datasink

In [8]:
lv1_workflow.connect([(preproc, datasink, [('outputspec.motion_parameters', 'report'),
                                          ('outputspec.motion_plots', 'report.@plot'),
                                          ('outputspec.realigned_files', 'bold.realign'),
                                          ('outputspec.smoothed_files', 'bold.smooth'),
                                          ('outputspec.highpassed_files', 'bold.highpass'),
                                          ('outputspec.reg_file', 'fs.@registration'),
                                          ('outputspec.reg_cost', 'fs.@reg_cost'),
                                          ('outputspec.reference', 'qa.@reference'),
                                          ('outputspec.mask_file', 'mask')]),
                     (lvl1model, datasink, [('outputspec.zfiles', '1stlevel'),
                                           ('outputspec.parameter_estimates', 
                                            '1stlevel.@param')]),
                      (fixedfx, datasink, [('outputspec.zstats', 'fixedfx.@zstats'),
                                          ('outputspec.copes', 'fixedfx.@con'),
                                          ('outputspec.varcopes', 'fixedfx.@variance'),
                                          ('outputspec.tstats', 'fixedfx.@tstats'),
                                          ('outputspec.res4d', 'fixedfx.@residuals')])
                     ])

### Run workflow!

In [9]:
lv1_workflow.run()

/Users/sll-members/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/Users/sll-members/anaconda/lib/python2.7/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)
/Users/sll-members/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/sll-members/fmri/Psych_204b/kalanit_fsl/tmp/metaflow/modelspec/art.07_1_BOLD_EPI_24mm_2sec_Accel2_reoriented_roi_dtype_mcf_outliers.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/Users/sll-members/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/sll-membe

# Classifier

In [10]:
preproc.outputs.outputspec


highpassed_files = None
mask_file = None
motion_parameters = None
motion_plots = None
realigned_files = None
reference = None
reg_cost = None
reg_file = None
smoothed_files = None